In [87]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from ipywidgets import interact, FloatSlider

In [88]:
def plot_normal_pdf(mu=0.0, sigma=1.0):
    x = np.linspace(mu - 5*sigma, mu + 5*sigma, 1000)
    y = norm.pdf(x, mu, sigma)
    
    plt.figure(figsize=(8, 5))
    plt.plot(x, y, label=f'μ = {mu}, σ = {sigma}')
    plt.axvline(mu, color='red', linestyle='--', label='Mean (μ)')
    plt.xlabel('x')
    plt.ylabel('f(x)')
    plt.title('Interactive Normal Distribution PDF')
    plt.grid(True)
    plt.minorticks_on()
    plt.legend()
    plt.show()

interact(plot_normal_pdf,
         mu=FloatSlider(value=0.0, min=-50.0, max=50.0, step=0.01, description='μ (mean)'),
         sigma=FloatSlider(value=0.01, min=0.01, max=50.0, step=0.01, description='σ (std)'))

interactive(children=(FloatSlider(value=0.0, description='μ (mean)', max=50.0, min=-50.0, step=0.01), FloatSli…

<function __main__.plot_normal_pdf(mu=0.0, sigma=1.0)>